In [1]:
# https://archive.ics.uci.edu/ml/datasets/opportunity+activity+recognition

In [2]:
import os
home_url = os.getcwd()
directory = "POC"

In [3]:
import sys
sys.path.append(home_url+'/../../Packages/Utils/')
sys.path.append(home_url+'/../../Packages/Model/')

In [4]:
# import sys
# sys.path.append(home_url+'/Downloads/'+directory+'/Debug-Packages/Utils/')
import matplotlib as plt
import pandas as pd
import os
import numpy as np
import math
import csv
from Visualization import plot_activity
from DataPreprocess import standardization_and_normalization
from sklearn import preprocessing
import tqdm

In [5]:
win_size=128
overlap = 0.90
activity_num = 4
source1 = 0
source2 = 1
target = 2

In [6]:
url = home_url+"/../../Dataset/OpportunityUCIDataset/dataset/"

In [7]:
# save_path = "/home/avijoy/Downloads/DTCN-AR/Preprocessing/OPPORTUNITY/ACC-Position-Preprocessed/"

In [8]:
step_size=int(win_size*overlap)
activity_num = 4
selected_user = 1
train_percentage = 0.6
valid_percentage = 0.2

AXIS = 3
FROM = 0
TO = FROM+3
START = 3
END = 4

In [9]:
item = ["train","valid","test"]
user = ["U1","U2","U3","U4"]
position = ["BACK", "RUA", "RLA", "LUA","LLA"]

In [11]:
folder_name = str(activity_num)+ " Activity"+"_Window "+str(win_size)+ "_Overlap "+str(overlap)
save_path = home_url +"/Data Files/"+folder_name+"/"

if not os.path.exists(save_path):
    os.makedirs(save_path)

# Data File Structure and Information
    - Locomotion Annotation: Stand, Walk, Sit, Lie
    Column Information:
    1: Milisec
    38-46: Back IMU
    51-59: Right Upper Arm
    64-72: Right Lower Arm
    77-85: Left Upper Arm
    90-98: Left Lower Arm
    103-117: Left Shoe
    119-133: Right Shoe
    244: Locomotion
    
    Shoe accumulated data is different than the Back, Left, right arm. Leave shoe data seperate.
    
    
    Locomotion Information
    Locomotion-1   -   Stand
    Locomotion-2   -   Walk
    Locomotion-3   -   Sit
    Locomotion-4   -   Lie
    
    Label Mapping
    - Stand: 1
    - Walk: 3
    - Sit: 0
    - Lie: 2

In [13]:
total_frame = {}
BACK_frame = {}
RUA_frame = {}
RLA_frame = {}
LUA_frame = {}
LLA_frame = {}

In [14]:
# Back_frame, RUA_frame, RLA_frame, LUA_frame, LLA_frame: 
# 11-15: User1 : For all 5 Frames
# 21-25: User2
# 31-35: User3
# 41-45: User4

In [15]:
# def plot_activity(df, position='Default'):
#     data = df[['Acc_x', 'Acc_y','Acc_z']][:df.shape[0]]

# #     position += '  Index Range: '+str(df.index[0])+ '  to  '+ str(df.index[-1])
#     axis = data.plot(subplots=True, figsize=(16, 12), title=position)
#     for ax in axis:
#         ax.legend(loc='lower left', bbox_to_anchor=(1.0, 0.5))
# #         ax.figure.savefig(visualization_url+position+'.png')

In [16]:
for user_index in range(1,5):
    for run_index in range(1,6):
        file_name = "S"+str(user_index)+"-ADL"+str(run_index)+".dat"
        dataframe = pd.read_csv(url+file_name,sep=" ", header=None) 
        
        
        ## Print the signal
        dataframe.sort_values(0)
        ## Print the signal
        index = (user_index * 10) + run_index
        
        BACK_IMU_frame = dataframe[[37,38,39,243]].copy()
        BACK_IMU_frame.rename(columns={37: 'Acc_x', 38:'Acc_y', 39: 'Acc_z', 243: 'gt'}, inplace=True)
        
        # Check the shape
        BACK_IMU_frame = BACK_IMU_frame.dropna()
        # Check the shape
        BACK_IMU_frame = BACK_IMU_frame[BACK_IMU_frame["gt"] != 0].copy()
        
        BACK_IMU_frame.loc[:, 'Person'] = user_index
        BACK_IMU_frame.loc[:, 'Position'] = "Back"
        BACK_frame[index] = BACK_IMU_frame.copy()
        BACK_frame[index].reset_index(drop=True, inplace=True)
        # Check the size

        
        RUA_IMU_frame = dataframe[[50,51,52,243]].copy()
        RUA_IMU_frame.rename(columns={50: 'Acc_x', 51:'Acc_y', 52: 'Acc_z', 243: 'gt'}, inplace=True)
        RUA_IMU_frame = RUA_IMU_frame.dropna()
        RUA_IMU_frame = RUA_IMU_frame[RUA_IMU_frame["gt"] != 0].copy()
        
        RUA_IMU_frame.loc[:, 'Person'] = user_index
        RUA_IMU_frame.loc[:, 'Position'] = "Right Upper Arm"
        RUA_frame[index] = RUA_IMU_frame.copy()
        RUA_frame[index].reset_index(drop=True, inplace=True)
        

        RLA_IMU_frame = dataframe[[63,64,65,243]].copy()
        RLA_IMU_frame.rename(columns={63: 'Acc_x', 64:'Acc_y', 65: 'Acc_z', 243: 'gt'}, inplace=True)
        RLA_IMU_frame = RLA_IMU_frame.dropna()
        RLA_IMU_frame = RLA_IMU_frame[RLA_IMU_frame["gt"] != 0].copy()

        RLA_IMU_frame.loc[:, 'Person'] = user_index
        RLA_IMU_frame.loc[:, 'Position'] = "Right Lower Arm"
        RLA_frame[index] = RLA_IMU_frame.copy()
        RLA_frame[index].reset_index(drop=True, inplace=True)
        
        
        
        LUA_IMU_frame = dataframe[[76,77,78,243]].copy()
        LUA_IMU_frame.rename(columns={76: 'Acc_x', 77:'Acc_y', 78: 'Acc_z', 243: 'gt'}, inplace=True)
        LUA_IMU_frame = LUA_IMU_frame.dropna()
        LUA_IMU_frame = LUA_IMU_frame[LUA_IMU_frame["gt"] != 0].copy()

        LUA_IMU_frame.loc[:, 'Person'] = user_index
        LUA_IMU_frame.loc[:, 'Position'] = "Left Upper Arm"
        LUA_frame[index] = LUA_IMU_frame.copy()
        LUA_frame[index].reset_index(drop=True, inplace=True)
        
        
        
        LLA_IMU_frame = dataframe[[89,90,91,243]].copy()
        LLA_IMU_frame.rename(columns={89: 'Acc_x', 90:'Acc_y', 91: 'Acc_z', 243: 'gt'}, inplace=True)
        LLA_IMU_frame = LLA_IMU_frame.dropna()
        LLA_IMU_frame = LLA_IMU_frame[LLA_IMU_frame["gt"] != 0].copy()

        LLA_IMU_frame.loc[:, 'Person'] = user_index
        LLA_IMU_frame.loc[:, 'Position'] = "Left Lower Arm"
        LLA_frame[index] = LLA_IMU_frame.copy()
        LLA_frame[index].reset_index(drop=True, inplace=True)
        

In [19]:
# BACK_frame[15].head()

In [23]:
# BACK_frame[15]['gt'].unique()

#### Ground truth has to start from 0

In [22]:
np.unique(BACK_frame[11]['gt'])

array([1, 2, 4, 5])

In [24]:
User1_Back_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User1_RUA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User1_RLA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User1_LUA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User1_LLA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])

User2_Back_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User2_RUA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User2_RLA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User2_LUA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User2_LLA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])

User3_Back_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User3_RUA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User3_RLA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User3_LUA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User3_LLA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])

User4_Back_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User4_RUA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User4_RLA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User4_LUA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
User4_LLA_frame_total = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])

In [25]:
back_shape = 0
for user_index in range(1,5):
    for run_index in range(1,6):
        index = (user_index * 10) + run_index

        if user_index == 1:
            User1_Back_frame_total = User1_Back_frame_total.append(BACK_frame[index], ignore_index=True)
            User1_RUA_frame_total = User1_RUA_frame_total.append(RUA_frame[index], ignore_index=True)
            User1_RLA_frame_total = User1_RLA_frame_total.append(RLA_frame[index], ignore_index=True)
            User1_LUA_frame_total = User1_LUA_frame_total.append(LUA_frame[index], ignore_index=True)
            User1_LLA_frame_total = User1_LLA_frame_total.append(LLA_frame[index], ignore_index=True)
        elif user_index == 2:
            User2_Back_frame_total = User2_Back_frame_total.append(BACK_frame[index], ignore_index=True)
            User2_RUA_frame_total = User2_RUA_frame_total.append(RUA_frame[index], ignore_index=True)
            User2_RLA_frame_total = User2_RLA_frame_total.append(RLA_frame[index], ignore_index=True)
            User2_LUA_frame_total = User2_LUA_frame_total.append(LUA_frame[index], ignore_index=True)
            User2_LLA_frame_total = User2_LLA_frame_total.append(LLA_frame[index], ignore_index=True)
        elif user_index == 3:
            User3_Back_frame_total = User3_Back_frame_total.append(BACK_frame[index], ignore_index=True)
            User3_RUA_frame_total = User3_RUA_frame_total.append(RUA_frame[index], ignore_index=True)
            User3_RLA_frame_total = User3_RLA_frame_total.append(RLA_frame[index], ignore_index=True)
            User3_LUA_frame_total = User3_LUA_frame_total.append(LUA_frame[index], ignore_index=True)
            User3_LLA_frame_total = User3_LLA_frame_total.append(LLA_frame[index], ignore_index=True)
        elif user_index == 4:
            User4_Back_frame_total = User4_Back_frame_total.append(BACK_frame[index], ignore_index=True)
            User4_RUA_frame_total = User4_RUA_frame_total.append(RUA_frame[index], ignore_index=True)
            User4_RLA_frame_total = User4_RLA_frame_total.append(RLA_frame[index], ignore_index=True)
            User4_LUA_frame_total = User4_LUA_frame_total.append(LUA_frame[index], ignore_index=True)
            User4_LLA_frame_total = User4_LLA_frame_total.append(LLA_frame[index], ignore_index=True)

In [26]:
np.unique(User1_Back_frame_total['gt'])

array([1, 2, 4, 5], dtype=object)

#### User 1 - Label Mapping

In [27]:
User1_Back_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User1_Back_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User1_RUA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User1_RUA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User1_RLA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User1_RLA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User1_LUA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User1_LUA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User1_LLA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User1_LLA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

In [28]:
np.unique(User1_LLA_frame_total['gt'])

array([0, 1, 2, 3])

#### User 2 - Label Mapping

In [29]:
User2_Back_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User2_Back_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User2_RUA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User2_RUA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User2_RLA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User2_RLA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User2_LUA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User2_LUA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User2_LLA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User2_LLA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

#### User 3 - Label Mapping

In [30]:
User3_Back_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User3_Back_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User3_RUA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User3_RUA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User3_RLA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User3_RLA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User3_LUA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User3_LUA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User3_LLA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User3_LLA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

#### User 4 - Label Mapping

In [31]:
User4_Back_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User4_Back_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User4_RUA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User4_RUA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User4_RLA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User4_RLA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User4_LUA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User4_LUA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

User4_LLA_frame_total.replace({'gt' : { 1:100, 2:200, 4:400, 5:500}}, inplace = True)
User4_LLA_frame_total.replace({'gt' : { 100:1, 200:3, 400:0, 500:2}}, inplace = True)

### User Specific, Position-wise Standardization and Normalization

#### User 1 - Back

In [32]:
person_gt = np.array(User1_Back_frame_total['Person'])
gt = np.array(User1_Back_frame_total['gt'])
position_gt = np.array(User1_Back_frame_total['Position'])

User1_Back_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User1_Back_frame_total.columns

np_scaled = standardization_and_normalization(User1_Back_frame_total)
User1_Back_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User1_Back_frame_total["Person"] = person_gt
User1_Back_frame_total["gt"] = gt
User1_Back_frame_total["Position"] = position_gt


In [33]:
# np.unique(User1_Back_frame_total['gt'])

In [34]:
# np.unique(User1_Back_frame_total['Person'])

In [35]:
# np.unique(User1_Back_frame_total['Position'])

#### User 2 - Back

In [36]:
person_gt = np.array(User2_Back_frame_total['Person'])
gt = np.array(User2_Back_frame_total['gt'])
position_gt = np.array(User2_Back_frame_total['Position'])

User2_Back_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User2_Back_frame_total.columns

np_scaled = standardization_and_normalization(User2_Back_frame_total)
User2_Back_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User2_Back_frame_total["Person"] = person_gt
User2_Back_frame_total["gt"] = gt
User2_Back_frame_total["Position"] = position_gt

#### User 3 - Back

In [37]:
person_gt = np.array(User3_Back_frame_total['Person'])
gt = np.array(User3_Back_frame_total['gt'])
position_gt = np.array(User3_Back_frame_total['Position'])

User3_Back_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User3_Back_frame_total.columns

np_scaled = standardization_and_normalization(User3_Back_frame_total)
User3_Back_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User3_Back_frame_total["Person"] = person_gt
User3_Back_frame_total["gt"] = gt
User3_Back_frame_total["Position"] = position_gt

#### User 4 - Back

In [38]:
person_gt = np.array(User4_Back_frame_total['Person'])
gt = np.array(User4_Back_frame_total['gt'])
position_gt = np.array(User4_Back_frame_total['Position'])

User4_Back_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User4_Back_frame_total.columns

np_scaled = standardization_and_normalization(User4_Back_frame_total)
User4_Back_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User4_Back_frame_total["Person"] = person_gt
User4_Back_frame_total["gt"] = gt
User4_Back_frame_total["Position"] = position_gt

In [39]:
# User4_Back_frame_total.head()

In [40]:
# np.max(User4_Back_frame_total['Acc_x'])

#### User1-RUA

In [41]:
person_gt = np.array(User1_RUA_frame_total['Person'])
gt = np.array(User1_RUA_frame_total['gt'])
position_gt = np.array(User1_RUA_frame_total['Position'])

User1_RUA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User1_RUA_frame_total.columns

np_scaled = standardization_and_normalization(User1_RUA_frame_total)
User1_RUA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User1_RUA_frame_total["Person"] = person_gt
User1_RUA_frame_total["gt"] = gt
User1_RUA_frame_total["Position"] = position_gt

#### User2 - RUA

In [42]:
person_gt = np.array(User2_RUA_frame_total['Person'])
gt = np.array(User2_RUA_frame_total['gt'])
position_gt = np.array(User2_RUA_frame_total['Position'])

User2_RUA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User2_RUA_frame_total.columns

np_scaled = standardization_and_normalization(User2_RUA_frame_total)
User2_RUA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User2_RUA_frame_total["Person"] = person_gt
User2_RUA_frame_total["gt"] = gt
User2_RUA_frame_total["Position"] = position_gt

#### User3 - RUA

In [43]:
person_gt = np.array(User3_RUA_frame_total['Person'])
gt = np.array(User3_RUA_frame_total['gt'])
position_gt = np.array(User3_RUA_frame_total['Position'])

User3_RUA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User3_RUA_frame_total.columns

np_scaled = standardization_and_normalization(User3_RUA_frame_total)
User3_RUA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User3_RUA_frame_total["Person"] = person_gt
User3_RUA_frame_total["gt"] = gt
User3_RUA_frame_total["Position"] = position_gt

#### USer4 - RUA

In [44]:
person_gt = np.array(User4_RUA_frame_total['Person'])
gt = np.array(User4_RUA_frame_total['gt'])
position_gt = np.array(User4_RUA_frame_total['Position'])

User4_RUA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User4_RUA_frame_total.columns

np_scaled = standardization_and_normalization(User4_RUA_frame_total)
User4_RUA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User4_RUA_frame_total["Person"] = person_gt
User4_RUA_frame_total["gt"] = gt
User4_RUA_frame_total["Position"] = position_gt

#### User 1 - RLA

In [45]:
person_gt = np.array(User1_RLA_frame_total['Person'])
gt = np.array(User1_RLA_frame_total['gt'])
position_gt = np.array(User1_RLA_frame_total['Position'])

User1_RLA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User1_RLA_frame_total.columns

np_scaled = standardization_and_normalization(User1_RLA_frame_total)
User1_RLA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User1_RLA_frame_total["Person"] = person_gt
User1_RLA_frame_total["gt"] = gt
User1_RLA_frame_total["Position"] = position_gt

#### User 2 - RLA

In [46]:
person_gt = np.array(User2_RLA_frame_total['Person'])
gt = np.array(User2_RLA_frame_total['gt'])
position_gt = np.array(User2_RLA_frame_total['Position'])

User2_RLA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User2_RLA_frame_total.columns

np_scaled = standardization_and_normalization(User2_RLA_frame_total)
User2_RLA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User2_RLA_frame_total["Person"] = person_gt
User2_RLA_frame_total["gt"] = gt
User2_RLA_frame_total["Position"] = position_gt

#### User 3 - RLA

In [47]:
person_gt = np.array(User3_RLA_frame_total['Person'])
gt = np.array(User3_RLA_frame_total['gt'])
position_gt = np.array(User3_RLA_frame_total['Position'])

User3_RLA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User3_RLA_frame_total.columns

np_scaled = standardization_and_normalization(User3_RLA_frame_total)
User3_RLA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User3_RLA_frame_total["Person"] = person_gt
User3_RLA_frame_total["gt"] = gt
User3_RLA_frame_total["Position"] = position_gt

#### User 4 - RLA

In [48]:
person_gt = np.array(User4_RLA_frame_total['Person'])
gt = np.array(User4_RLA_frame_total['gt'])
position_gt = np.array(User4_RLA_frame_total['Position'])

User4_RLA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User4_RLA_frame_total.columns

np_scaled = standardization_and_normalization(User4_RLA_frame_total)
User4_RLA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User4_RLA_frame_total["Person"] = person_gt
User4_RLA_frame_total["gt"] = gt
User4_RLA_frame_total["Position"] = position_gt

#### User 1 - LUA

In [49]:
person_gt = np.array(User1_LUA_frame_total['Person'])
gt = np.array(User1_LUA_frame_total['gt'])
position_gt = np.array(User1_LUA_frame_total['Position'])

User1_LUA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User1_LUA_frame_total.columns

np_scaled = standardization_and_normalization(User1_LUA_frame_total)
User1_LUA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User1_LUA_frame_total["Person"] = person_gt
User1_LUA_frame_total["gt"] = gt
User1_LUA_frame_total["Position"] = position_gt

#### User 2 - LUA

In [50]:
person_gt = np.array(User2_LUA_frame_total['Person'])
gt = np.array(User2_LUA_frame_total['gt'])
position_gt = np.array(User2_LUA_frame_total['Position'])

User2_LUA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User2_LUA_frame_total.columns

np_scaled = standardization_and_normalization(User2_LUA_frame_total)
User2_LUA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User2_LUA_frame_total["Person"] = person_gt
User2_LUA_frame_total["gt"] = gt
User2_LUA_frame_total["Position"] = position_gt

#### User 3 - LUA

In [51]:
person_gt = np.array(User3_LUA_frame_total['Person'])
gt = np.array(User3_LUA_frame_total['gt'])
position_gt = np.array(User3_LUA_frame_total['Position'])

User3_LUA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User3_LUA_frame_total.columns

np_scaled = standardization_and_normalization(User3_LUA_frame_total)
User3_LUA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User3_LUA_frame_total["Person"] = person_gt
User3_LUA_frame_total["gt"] = gt
User3_LUA_frame_total["Position"] = position_gt

#### User 4 - LUA

In [52]:
person_gt = np.array(User4_LUA_frame_total['Person'])
gt = np.array(User4_LUA_frame_total['gt'])
position_gt = np.array(User4_LUA_frame_total['Position'])

User4_LUA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User4_LUA_frame_total.columns

np_scaled = standardization_and_normalization(User4_LUA_frame_total)
User4_LUA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User4_LUA_frame_total["Person"] = person_gt
User4_LUA_frame_total["gt"] = gt
User4_LUA_frame_total["Position"] = position_gt

#### User 1 - LLA

In [53]:
person_gt = np.array(User1_LLA_frame_total['Person'])
gt = np.array(User1_LLA_frame_total['gt'])
position_gt = np.array(User1_LLA_frame_total['Position'])

User1_LLA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User1_LLA_frame_total.columns

np_scaled = standardization_and_normalization(User1_LLA_frame_total)
User1_LLA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User1_LLA_frame_total["Person"] = person_gt
User1_LLA_frame_total["gt"] = gt
User1_LLA_frame_total["Position"] = position_gt

#### User 2 - LLA

In [54]:
person_gt = np.array(User2_LLA_frame_total['Person'])
gt = np.array(User2_LLA_frame_total['gt'])
position_gt = np.array(User2_LLA_frame_total['Position'])

User2_LLA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User2_LLA_frame_total.columns

np_scaled = standardization_and_normalization(User2_LLA_frame_total)
User2_LLA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User2_LLA_frame_total["Person"] = person_gt
User2_LLA_frame_total["gt"] = gt
User2_LLA_frame_total["Position"] = position_gt

#### User 3 - LLA

In [55]:
person_gt = np.array(User3_LLA_frame_total['Person'])
gt = np.array(User3_LLA_frame_total['gt'])
position_gt = np.array(User3_LLA_frame_total['Position'])

User3_LLA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User3_LLA_frame_total.columns

np_scaled = standardization_and_normalization(User3_LLA_frame_total)
User3_LLA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User3_LLA_frame_total["Person"] = person_gt
User3_LLA_frame_total["gt"] = gt
User3_LLA_frame_total["Position"] = position_gt

#### User 4 - LLA

In [56]:
person_gt = np.array(User4_LLA_frame_total['Person'])
gt = np.array(User4_LLA_frame_total['gt'])
position_gt = np.array(User4_LLA_frame_total['Position'])

User4_LLA_frame_total.drop(['Person','gt','Position'], axis=1, inplace=True)
column_name = User4_LLA_frame_total.columns

np_scaled = standardization_and_normalization(User4_LLA_frame_total)
User4_LLA_frame_total = pd.DataFrame(np_scaled, columns=column_name)

User4_LLA_frame_total["Person"] = person_gt
User4_LLA_frame_total["gt"] = gt
User4_LLA_frame_total["Position"] = position_gt

In [57]:
# User1_LLA_frame_total.head()

In [58]:
# User1_LLA_frame_total.tail()

## Train-Valid-Test Split

#### User 1

In [59]:
activity_set = np.unique(User1_Back_frame_total['gt'])
total_activity = len(np.unique(User1_Back_frame_total['gt']))

In [60]:
total_activity

4

In [61]:
activity_set

array([0, 1, 2, 3])

In [62]:
user1_back_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user1_back_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user1_back_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    
    activity_frame = User1_Back_frame_total[User1_Back_frame_total['gt'] == activity_set[activity_index]]

    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user1_back_train = user1_back_train.append(train, ignore_index=True, sort = False)
    user1_back_valid = user1_back_valid.append(valid, ignore_index=True, sort = False)
    user1_back_test = user1_back_test.append(test, ignore_index=True, sort = False)
    
user1_back_train.to_csv (save_path+'U1_BACK_train'+'.csv', index = None, header=True)
user1_back_valid.to_csv (save_path+'U1_BACK_valid'+'.csv', index = None, header=True)
user1_back_test.to_csv (save_path+'U1_BACK_test'+'.csv', index = None, header=True)


# plot_activity(user1_back_train, "Default")
# plot_activity(user1_back_valid, "Default")

In [63]:
activity_set = np.unique(User1_RUA_frame_total['gt'])
total_activity = len(np.unique(User1_RUA_frame_total['gt']))

In [64]:
total_activity

4

In [65]:
user1_RUA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user1_RUA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user1_RUA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User1_RUA_frame_total[User1_RUA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user1_RUA_train = user1_RUA_train.append(train, ignore_index=True, sort = False)
    user1_RUA_valid = user1_RUA_valid.append(valid, ignore_index=True, sort = False)
    user1_RUA_test = user1_RUA_test.append(test, ignore_index=True, sort = False)
    
user1_RUA_train.to_csv (save_path+'U1_RUA_train'+'.csv', index = None, header=True)
user1_RUA_valid.to_csv (save_path+'U1_RUA_valid'+'.csv', index = None, header=True)
user1_RUA_test.to_csv (save_path+'U1_RUA_test'+'.csv', index = None, header=True)

In [66]:
activity_set = np.unique(User1_RLA_frame_total['gt'])
total_activity = len(np.unique(User1_RLA_frame_total['gt']))

In [67]:
total_activity

4

In [68]:
user1_RLA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user1_RLA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user1_RLA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User1_RLA_frame_total[User1_RLA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user1_RLA_train = user1_RLA_train.append(train, ignore_index=True, sort = False)
    user1_RLA_valid = user1_RLA_valid.append(valid, ignore_index=True, sort = False)
    user1_RLA_test = user1_RLA_test.append(test, ignore_index=True, sort = False)
    
user1_RLA_train.to_csv (save_path+'U1_RLA_train'+'.csv', index = None, header=True)
user1_RLA_valid.to_csv (save_path+'U1_RLA_valid'+'.csv', index = None, header=True)
user1_RLA_test.to_csv (save_path+'U1_RLA_test'+'.csv', index = None, header=True)

In [69]:
activity_set = np.unique(User1_LUA_frame_total['gt'])
total_activity = len(np.unique(User1_LUA_frame_total['gt']))

In [70]:
total_activity

4

In [71]:
user1_LUA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user1_LUA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user1_LUA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User1_LUA_frame_total[User1_LUA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user1_LUA_train = user1_LUA_train.append(train, ignore_index=True, sort = False)
    user1_LUA_valid = user1_LUA_valid.append(valid, ignore_index=True, sort = False)
    user1_LUA_test = user1_LUA_test.append(test, ignore_index=True, sort = False)
    
user1_LUA_train.to_csv (save_path+'U1_LUA_train'+'.csv', index = None, header=True)
user1_LUA_valid.to_csv (save_path+'U1_LUA_valid'+'.csv', index = None, header=True)
user1_LUA_test.to_csv (save_path+'U1_LUA_test'+'.csv', index = None, header=True)

In [72]:
activity_set = np.unique(User1_LLA_frame_total['gt'])
total_activity = len(np.unique(User1_LLA_frame_total['gt']))

In [73]:
total_activity

4

In [74]:
user1_LLA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user1_LLA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user1_LLA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User1_LLA_frame_total[User1_LLA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user1_LLA_train = user1_LLA_train.append(train, ignore_index=True, sort = False)
    user1_LLA_valid = user1_LLA_valid.append(valid, ignore_index=True, sort = False)
    user1_LLA_test = user1_LLA_test.append(test, ignore_index=True, sort = False)
    
user1_LLA_train.to_csv (save_path+'U1_LLA_train'+'.csv', index = None, header=True)
user1_LLA_valid.to_csv (save_path+'U1_LLA_valid'+'.csv', index = None, header=True)
user1_LLA_test.to_csv (save_path+'U1_LLA_test'+'.csv', index = None, header=True)

#### User 2

In [75]:
activity_set = np.unique(User2_Back_frame_total['gt'])
total_activity = len(np.unique(User2_Back_frame_total['gt']))

In [76]:
total_activity

4

In [77]:
user2_back_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user2_back_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user2_back_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User2_Back_frame_total[User2_Back_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user2_back_train = user2_back_train.append(train, ignore_index=True, sort = False)
    user2_back_valid = user2_back_valid.append(valid, ignore_index=True, sort = False)
    user2_back_test = user2_back_test.append(test, ignore_index=True, sort = False)
    
    
user2_back_train.to_csv (save_path+'U2_BACK_train'+'.csv', index = None, header=True)
user2_back_valid.to_csv (save_path+'U2_BACK_valid'+'.csv', index = None, header=True)
user2_back_test.to_csv (save_path+'U2_BACK_test'+'.csv', index = None, header=True)

In [78]:
activity_set = np.unique(User2_RUA_frame_total['gt'])
total_activity = len(np.unique(User2_RUA_frame_total['gt']))

In [79]:
total_activity

4

In [80]:
user2_RUA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user2_RUA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user2_RUA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User2_RUA_frame_total[User2_RUA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user2_RUA_train = user2_RUA_train.append(train, ignore_index=True, sort = False)
    user2_RUA_valid = user2_RUA_valid.append(valid, ignore_index=True, sort = False)
    user2_RUA_test = user2_RUA_test.append(test, ignore_index=True, sort = False)
    
user2_RUA_train.to_csv (save_path+'U2_RUA_train'+'.csv', index = None, header=True)
user2_RUA_valid.to_csv (save_path+'U2_RUA_valid'+'.csv', index = None, header=True)
user2_RUA_test.to_csv (save_path+'U2_RUA_test'+'.csv', index = None, header=True)

In [81]:
activity_set = np.unique(User2_RLA_frame_total['gt'])
total_activity = len(np.unique(User2_RLA_frame_total['gt']))

In [82]:
total_activity

4

In [83]:
user2_RLA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user2_RLA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user2_RLA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User2_RLA_frame_total[User2_RLA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user2_RLA_train = user2_RLA_train.append(train, ignore_index=True, sort = False)
    user2_RLA_valid = user2_RLA_valid.append(valid, ignore_index=True, sort = False)
    user2_RLA_test = user2_RLA_test.append(test, ignore_index=True, sort = False)
    
user2_RLA_train.to_csv (save_path+'U2_RLA_train'+'.csv', index = None, header=True)
user2_RLA_valid.to_csv (save_path+'U2_RLA_valid'+'.csv', index = None, header=True)
user2_RLA_test.to_csv (save_path+'U2_RLA_test'+'.csv', index = None, header=True)

In [84]:
activity_set = np.unique(User2_LUA_frame_total['gt'])
total_activity = len(np.unique(User2_LUA_frame_total['gt']))

In [85]:
total_activity

4

In [86]:
user2_LUA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user2_LUA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user2_LUA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User2_LUA_frame_total[User2_LUA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user2_LUA_train = user2_LUA_train.append(train, ignore_index=True, sort = False)
    user2_LUA_valid = user2_LUA_valid.append(valid, ignore_index=True, sort = False)
    user2_LUA_test = user2_LUA_test.append(test, ignore_index=True, sort = False)
    
user2_LUA_train.to_csv (save_path+'U2_LUA_train'+'.csv', index = None, header=True)
user2_LUA_valid.to_csv (save_path+'U2_LUA_valid'+'.csv', index = None, header=True)
user2_LUA_test.to_csv (save_path+'U2_LUA_test'+'.csv', index = None, header=True)

In [87]:
activity_set = np.unique(User2_LLA_frame_total['gt'])
total_activity = len(np.unique(User2_LLA_frame_total['gt']))

In [88]:
total_activity

4

In [89]:
user2_LLA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user2_LLA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user2_LLA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User2_LLA_frame_total[User2_LLA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user2_LLA_train = user2_LLA_train.append(train, ignore_index=True, sort = False)
    user2_LLA_valid = user2_LLA_valid.append(valid, ignore_index=True, sort = False)
    user2_LLA_test = user2_LLA_test.append(test, ignore_index=True, sort = False)
    
user2_LLA_train.to_csv (save_path+'U2_LLA_train'+'.csv', index = None, header=True)
user2_LLA_valid.to_csv (save_path+'U2_LLA_valid'+'.csv', index = None, header=True)
user2_LLA_test.to_csv (save_path+'U2_LLA_test'+'.csv', index = None, header=True)

#### User 3

In [90]:
activity_set = np.unique(User3_Back_frame_total['gt'])
total_activity = len(np.unique(User3_Back_frame_total['gt']))

In [91]:
total_activity

4

In [92]:
user3_back_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user3_back_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user3_back_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User3_Back_frame_total[User3_Back_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user3_back_train = user3_back_train.append(train, ignore_index=True, sort = False)
    user3_back_valid = user3_back_valid.append(valid, ignore_index=True, sort = False)
    user3_back_test = user3_back_test.append(test, ignore_index=True, sort = False)

user3_back_train.to_csv (save_path+'U3_BACK_train'+'.csv', index = None, header=True)
user3_back_valid.to_csv (save_path+'U3_BACK_valid'+'.csv', index = None, header=True)
user3_back_test.to_csv (save_path+'U3_BACK_test'+'.csv', index = None, header=True)

In [93]:
activity_set = np.unique(User3_RUA_frame_total['gt'])
total_activity = len(np.unique(User3_RUA_frame_total['gt']))

In [94]:
total_activity

4

In [95]:
user3_RUA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user3_RUA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user3_RUA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User3_RUA_frame_total[User3_RUA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user3_RUA_train = user3_RUA_train.append(train, ignore_index=True, sort = False)
    user3_RUA_valid = user3_RUA_valid.append(valid, ignore_index=True, sort = False)
    user3_RUA_test = user3_RUA_test.append(test, ignore_index=True, sort = False)
    
user3_RUA_train.to_csv (save_path+'U3_RUA_train'+'.csv', index = None, header=True)
user3_RUA_valid.to_csv (save_path+'U3_RUA_valid'+'.csv', index = None, header=True)
user3_RUA_test.to_csv (save_path+'U3_RUA_test'+'.csv', index = None, header=True)

In [96]:
activity_set = np.unique(User3_RLA_frame_total['gt'])
total_activity = len(np.unique(User3_RLA_frame_total['gt']))

In [97]:
total_activity

4

In [98]:
user3_RLA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user3_RLA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user3_RLA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User3_RLA_frame_total[User3_RLA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user3_RLA_train = user3_RLA_train.append(train, ignore_index=True, sort = False)
    user3_RLA_valid = user3_RLA_valid.append(valid, ignore_index=True, sort = False)
    user3_RLA_test = user3_RLA_test.append(test, ignore_index=True, sort = False)
    
user3_RLA_train.to_csv (save_path+'U3_RLA_train'+'.csv', index = None, header=True)
user3_RLA_valid.to_csv (save_path+'U3_RLA_valid'+'.csv', index = None, header=True)
user3_RLA_test.to_csv (save_path+'U3_RLA_test'+'.csv', index = None, header=True)

In [99]:
activity_set = np.unique(User3_LUA_frame_total['gt'])
total_activity = len(np.unique(User3_LUA_frame_total['gt']))

In [100]:
total_activity

4

In [101]:
user3_LUA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user3_LUA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user3_LUA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User3_LUA_frame_total[User3_LUA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user3_LUA_train = user3_LUA_train.append(train, ignore_index=True, sort = False)
    user3_LUA_valid = user3_LUA_valid.append(valid, ignore_index=True, sort = False)
    user3_LUA_test = user3_LUA_test.append(test, ignore_index=True, sort = False)
    
user3_LUA_train.to_csv (save_path+'U3_LUA_train'+'.csv', index = None, header=True)
user3_LUA_valid.to_csv (save_path+'U3_LUA_valid'+'.csv', index = None, header=True)
user3_LUA_test.to_csv (save_path+'U3_LUA_test'+'.csv', index = None, header=True)

In [102]:
activity_set = np.unique(User3_LLA_frame_total['gt'])
total_activity = len(np.unique(User3_LLA_frame_total['gt']))

In [103]:
total_activity

4

In [104]:
user3_LLA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user3_LLA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user3_LLA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User3_LLA_frame_total[User3_LLA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user3_LLA_train = user3_LLA_train.append(train, ignore_index=True, sort = False)
    user3_LLA_valid = user3_LLA_valid.append(valid, ignore_index=True, sort = False)
    user3_LLA_test = user3_LLA_test.append(test, ignore_index=True, sort = False)
    
user3_LLA_train.to_csv (save_path+'U3_LLA_train'+'.csv', index = None, header=True)
user3_LLA_valid.to_csv (save_path+'U3_LLA_valid'+'.csv', index = None, header=True)
user3_LLA_test.to_csv (save_path+'U3_LLA_test'+'.csv', index = None, header=True)

#### User 4

In [105]:
activity_set = np.unique(User4_Back_frame_total['gt'])
total_activity = len(np.unique(User4_Back_frame_total['gt']))

In [106]:
total_activity

4

In [107]:
user4_back_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user4_back_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user4_back_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User4_Back_frame_total[User4_Back_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user4_back_train = user4_back_train.append(train, ignore_index=True, sort = False)
    user4_back_valid = user4_back_valid.append(valid, ignore_index=True, sort = False)
    user4_back_test = user4_back_test.append(test, ignore_index=True, sort = False)
    
user4_back_train.to_csv (save_path+'U4_BACK_train'+'.csv', index = None, header=True)
user4_back_valid.to_csv (save_path+'U4_BACK_valid'+'.csv', index = None, header=True)
user4_back_test.to_csv (save_path+'U4_BACK_test'+'.csv', index = None, header=True)

In [108]:
activity_set = np.unique(User4_RUA_frame_total['gt'])
total_activity = len(np.unique(User4_RUA_frame_total['gt']))

In [109]:
total_activity

4

In [110]:
user4_RUA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user4_RUA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user4_RUA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User4_RUA_frame_total[User4_RUA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user4_RUA_train = user4_RUA_train.append(train, ignore_index=True, sort = False)
    user4_RUA_valid = user4_RUA_valid.append(valid, ignore_index=True, sort = False)
    user4_RUA_test = user4_RUA_test.append(test, ignore_index=True, sort = False)
    
user4_RUA_train.to_csv (save_path+'U4_RUA_train'+'.csv', index = None, header=True)
user4_RUA_valid.to_csv (save_path+'U4_RUA_valid'+'.csv', index = None, header=True)
user4_RUA_test.to_csv (save_path+'U4_RUA_test'+'.csv', index = None, header=True)

In [111]:
activity_set = np.unique(User4_RLA_frame_total['gt'])
total_activity = len(np.unique(User4_RLA_frame_total['gt']))

In [112]:
total_activity

4

In [113]:
user4_RLA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user4_RLA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user4_RLA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User4_RLA_frame_total[User4_RLA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user4_RLA_train = user4_RLA_train.append(train, ignore_index=True, sort = False)
    user4_RLA_valid = user4_RLA_valid.append(valid, ignore_index=True, sort = False)
    user4_RLA_test = user4_RLA_test.append(test, ignore_index=True, sort = False)
    
user4_RLA_train.to_csv (save_path+'U4_RLA_train'+'.csv', index = None, header=True)
user4_RLA_valid.to_csv (save_path+'U4_RLA_valid'+'.csv', index = None, header=True)
user4_RLA_test.to_csv (save_path+'U4_RLA_test'+'.csv', index = None, header=True)

In [114]:
activity_set = np.unique(User4_LUA_frame_total['gt'])
total_activity = len(np.unique(User4_LUA_frame_total['gt']))

In [115]:
total_activity

4

In [116]:
user4_LUA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user4_LUA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user4_LUA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User4_LUA_frame_total[User4_LUA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user4_LUA_train = user4_LUA_train.append(train, ignore_index=True, sort = False)
    user4_LUA_valid = user4_LUA_valid.append(valid, ignore_index=True, sort = False)
    user4_LUA_test = user4_LUA_test.append(test, ignore_index=True, sort = False)
    
user4_LUA_train.to_csv (save_path+'U4_LUA_train'+'.csv', index = None, header=True)
user4_LUA_valid.to_csv (save_path+'U4_LUA_valid'+'.csv', index = None, header=True)
user4_LUA_test.to_csv (save_path+'U4_LUA_test'+'.csv', index = None, header=True)

In [117]:
activity_set = np.unique(User4_LLA_frame_total['gt'])
total_activity = len(np.unique(User4_LLA_frame_total['gt']))

In [118]:
total_activity

4

In [119]:
user4_LLA_train = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user4_LLA_valid = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
user4_LLA_test = pd.DataFrame(columns=['Acc_x', 'Acc_y', 'Acc_z', 'gt', 'Person', 'Position'])
for activity_index in range(0, total_activity):
    activity_frame = User4_LLA_frame_total[User4_LLA_frame_total['gt'] == activity_set[activity_index]]
    
    train_upto = int(activity_frame.shape[0]*train_percentage)    
    valid_upto = int(activity_frame.shape[0]*valid_percentage)  
        
    train = activity_frame[0:train_upto].copy()
    valid = activity_frame[train_upto+1 : train_upto+valid_upto].copy()
    test = activity_frame[train_upto+valid_upto+1 : activity_frame.shape[0]].copy()
    
    user4_LLA_train = user4_LLA_train.append(train, ignore_index=True, sort = False)
    user4_LLA_valid = user4_LLA_valid.append(valid, ignore_index=True, sort = False)
    user4_LLA_test = user4_LLA_test.append(test, ignore_index=True, sort = False)

user4_LLA_train.to_csv (save_path+'U4_LLA_train'+'.csv', index = None, header=True)
user4_LLA_valid.to_csv (save_path+'U4_LLA_valid'+'.csv', index = None, header=True)
user4_LLA_test.to_csv (save_path+'U4_LLA_test'+'.csv', index = None, header=True)

In [120]:
# np.unique(user4_LLA_train['gt'])

In [121]:
# np.unique(user4_LLA_valid['gt'])

In [122]:
# len(user4_LLA_valid)

### Code block for merging all user train and test data for a specific position

## Prepare Final Dataset

In [123]:
BACK_dataset_train = []
RUA_dataset_train = []
RLA_dataset_train = []
LUA_dataset_train = []
LLA_dataset_train = []
BACK_gt_train = []
RUA_gt_train = []
RLA_gt_train = []
LUA_gt_train = []
LLA_gt_train = []

In [124]:
BACK_dataset_valid = []
RUA_dataset_valid = []
RLA_dataset_valid = []
LUA_dataset_valid = []
LLA_dataset_valid = []
BACK_gt_valid = []
RUA_gt_valid = []
RLA_gt_valid = []
LUA_gt_valid = []
LLA_gt_valid = []

In [125]:
BACK_dataset_test = []
RUA_dataset_test = []
RLA_dataset_test = []
LUA_dataset_test = []
LLA_dataset_test = []
BACK_gt_test = []
RUA_gt_test = []
RLA_gt_test = []
LUA_gt_test = []
LLA_gt_test = []

### Code Block for taking position-wise all user train and validation data

In [ ]:
for position_index in tqdm.tqdm(range(0,5)): #Back, RUA, RLA, LUA, LLA
    for split_index in range(0,3):
        file_name = user[selected_user-1] + "_" + position[position_index]+'_'+item[split_index]
        
        print(file_name)
        df = pd.read_csv(save_path+file_name+'.csv', sep=",")   
        len_df = df.shape[0]
        narray = df.to_numpy()

        for i in range(0, len_df, step_size):
            window = narray[i:i+win_size, FROM:TO]
            
            if window.shape[0] != win_size:
                continue
            else:
                reshaped_window = window.reshape(1,win_size,1,AXIS)
                gt = np.bincount(narray[i:i+win_size,START:END].astype(int).ravel()).argmax()
                
                if position_index == 0:
                    if split_index == 0:
                        BACK_dataset_train.append(reshaped_window)
                        BACK_gt_train.append(gt)
                    elif split_index == 1:
                        BACK_dataset_valid.append(reshaped_window)
                        BACK_gt_valid.append(gt)
                    elif split_index == 2:
                        BACK_dataset_test.append(reshaped_window)
                        BACK_gt_test.append(gt)
                elif position_index == 1:
                    if split_index == 0:
                        RUA_dataset_train.append(reshaped_window)
                        RUA_gt_train.append(gt)
                    elif split_index == 1:
                        RUA_dataset_valid.append(reshaped_window)
                        RUA_gt_valid.append(gt)
                    elif split_index == 2:
                        RUA_dataset_test.append(reshaped_window)
                        RUA_gt_test.append(gt)
                elif position_index == 2:
                    if split_index == 0:
                        RLA_dataset_train.append(reshaped_window)
                        RLA_gt_train.append(gt)
                    elif split_index == 1:
                        RLA_dataset_valid.append(reshaped_window)
                        RLA_gt_valid.append(gt)
                    elif split_index == 2:
                        RLA_dataset_test.append(reshaped_window)
                        RLA_gt_test.append(gt)
                elif position_index == 3:
                    if split_index == 0:
                        LUA_dataset_train.append(reshaped_window)
                        LUA_gt_train.append(gt)
                    elif split_index == 1:
                        LUA_dataset_valid.append(reshaped_window)
                        LUA_gt_valid.append(gt)
                    elif split_index == 2:
                        LUA_dataset_test.append(reshaped_window)
                        LUA_gt_test.append(gt)
                elif position_index == 4:
                    if split_index == 0:
                        LLA_dataset_train.append(reshaped_window)
                        LLA_gt_train.append(gt)
                    elif split_index == 1:
                        LLA_dataset_valid.append(reshaped_window)
                        LLA_gt_valid.append(gt)
                    elif split_index == 2:
                        LLA_dataset_test.append(reshaped_window)
                        LLA_gt_test.append(gt)